In [5]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv
from scipy.stats import pearsonr
import numpy as np

%matplotlib inline


In [7]:
def get_exchange_reserves(start, end):
    load_dotenv()
    access_token = os.getenv("CQ_API_KEY")
    headers = { 'Authorization': 'Bearer ' + access_token }
    import urllib.request
    url_a = f"https://api.cryptoquant.com/v1/btc/exchange-flows/reserve?exchange=binance&window=day&from={start}&to={end}&limit=100000"
#only retrieves data as far back as 06/23/2017

    response_a = requests.get(url_a, headers=headers)
    all_exchange_reserve=response_a.json()
    print(type(all_exchange_reserve))
#pprint.pprint(all_exchange_reserve) to see it better

    all_exchange_df = pd.DataFrame(data=all_exchange_reserve['result']['data'])
    all_exchange_df['date'] = pd.to_datetime(all_exchange_df['date'])
    all_exchange_df = all_exchange_df.set_index('date')
    exchange_reserve_df=all_exchange_df.drop(columns=["reserve_usd"])
    
    return exchange_reserve_df

In [8]:
reserves_df=get_exchange_reserves(20170623,20200122 )
reserves_df.head()

<class 'dict'>


,reserve
date,
2020-01-22,248310.537097
2020-01-21,249435.697632
2020-01-20,251097.368547
2020-01-19,252877.237501
2020-01-18,250395.183134


In [9]:
def reserves_plot():
    fig_r= reserves_df.hvplot(title= "Reserve Bitcoin On Exchanges",xlabel="date", ylabel="Reserves In Bitcoin")
    
    return pn.panel(fig_r)

In [10]:
def get_moving_average():
    file_path=Path("..//data/clean_data/21wk_MA.csv")
    moving_average=pd.read_csv(file_path, header=0, index_col="date", parse_dates=True, infer_datetime_format=True)
    moving_average= moving_average.dropna()
    
    return moving_average

In [13]:
moving_average_df= pd.DataFrame(get_moving_average())
moving_average_df.head()

,price,21_WK_MA
date,,
2015-06-12,229.47,240.76
2015-06-19,246.33,242.19
2015-06-26,243.00,242.73
2015-07-03,256.95,243.39
2015-07-10,286.67,245.00


In [15]:
moving_average_df.corr()

,price,21_WK_MA
price,1.000000,0.865091
21_WK_MA,0.865091,1.000000


In [14]:
def get_ma_plot():
    fig_ma= moving_average_df.hvplot(title= "BTC Price vs 21 Week Moving Average",xlabel="Date", ylabel="Price In USD", legend=True)
    
    return pn.panel(fig_ma)